# Getting started with SplunkUtills

SplunkUtills is an API for Splunk database, which can be used for querying SIRI and GTFS data into Pandas DataFrame. 

## Step 1: Install  splunk-sdk

SplunkUtills requires installing splunk-sdk package.

You can use pip for installation.

In [1]:
!pip install splunk-sdk

## Step 2:  Set Splunk credentials

One must have Splunk credentials (host, port, username, password) to set a connection to Splunk using SplunkUtills.

Note that the port is Splunk management port and not web port.

You can save your credentials in json file as in the following example or simply hard-coded them.

In [2]:
import json

with open('credentials.json', 'r') as f:
    credentials = json.load(f)
    
HOST, PORT, USERNAME, PASSWORD = credentials.values()

## Step 3: Download & import SplunkUtills

Download SplunkUtills.py (needs to be located at the same folder as this notebook).

Then import **splunk_query_builder** and **read_splunk** from SplunkUtills.

In [3]:
from SplunkUtills import splunk_query_builder, read_splunk

## Step 4: Read Splunk function 

Args:
   * query (str): SPL query.
   * username (str): Splunk username.
   * password (str): Splunk password.
   * host (str): Splunk host.
   * port (int): Splunk management port.
   * time_limit (int): time limit (minutes) for the query (default: 5).
   
Returns:
    DataFrame: query results.
    
**For example lets query the GTFS (route_stats) using Splunk's Search Processing Language (SPL).**

In [4]:
%%time

query = '''search index=route_stats earliest=-12d latest=-10d route_id=5189|
        fields agency_id, route_short_name, route_id, date, all_start_time | '''

GTFS_data = read_splunk(query,
          host =  HOST, port = PORT, username = USERNAME, password = PASSWORD)

start..

your query:
 search index=route_stats earliest=-12d latest=-10d route_id=5189|

        fields agency_id, route_short_name, route_id, date, all_start_time |
 

connection succeed

query status: 100.0%   13 scanned   2 matched   2 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 2

Wall time: 4.55 s


In [5]:
GTFS_data.head()

,agency_id,route_short_name,route_id,date,all_start_time
0,3,501,5189,2019-05-24,05:30:00;05:45:00;05:55:00;06:05:00;06:15:00;0...
1,3,501,5189,2019-05-23,05:15:00;05:56:00;06:40:00;07:10:00;07:40:00;0...


**Note that read_splunk function runs a Splunk job. One might send a very heavy query/ies which can fail the connection.
You can stop and delete jobs using Splunk app Job Manager.**

## Step 5:  Splunk Query Builder function

To save time of learning the SPL syntax, you can use this function, which creates SPL query from Dictionary of query Kwargs.

**For example lets query SIRI using splunk_query_builder function.**


**First, we will need to set our query_kwargs for filtering by columns values.**

In [6]:
query_kwargs = {
                'index': 'siri',
                'earliest': '-10d', #=last 10 days
                #'latest': '-8d',
                #'agency_id': 3,
                'route_short_name': 501,
                'route_id': 5189,
                'planned_start_time': '22:00:00',
               #'max_columns': 100000,
               'columns': ['timestamp','agency_id','route_id','route_short_name','service_id','planned_start_time',
                        'bus_id','predicted_end_time','time_recorded','lat','lon']
                }

In [7]:
%%time

SIRI_data = read_splunk(splunk_query_builder(**query_kwargs),
          host =  HOST, port = PORT, username = USERNAME, password = PASSWORD)

start..

your query:
 search index=siri earliest=-10d route_short_name=501 route_id=5189 planned_start_time=22:00:00 |
 fields timestamp, agency_id, route_id, route_short_name, service_id, planned_start_time, bus_id, predicted_end_time, time_recorded, lat, lon |


connection succeed

query status: 100.0%   9739 scanned   938 matched   938 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 938

Wall time: 5.52 s


In [8]:
SIRI_data.head()

,timestamp,agency_id,route_id,route_short_name,service_id,planned_start_time,bus_id,predicted_end_time,time_recorded,lat,lon
0,2019-06-02T23:01:17.058,3,5189,501,39061043,22:00:00,7647969,23:02:00,23:01:26,32.177559,34.887798
1,2019-06-02T23:00:17.06,3,5189,501,39061043,22:00:00,7647969,23:02:00,23:00:42,32.178257,34.884609
2,2019-06-02T22:59:17.062,3,5189,501,39061043,22:00:00,7647969,23:02:00,22:59:43,32.179039,34.881069
3,2019-06-02T22:58:17.061,3,5189,501,39061043,22:00:00,7647969,23:01:00,22:58:20,32.179661,34.878082
4,2019-06-02T22:57:17.069,3,5189,501,39061043,22:00:00,7647969,23:01:00,22:57:44,32.179974,34.876816


**Some tips for splunk_query_builder function:**

* Splunk search works better when using indexes for filtering. As for now, the function gets only one value per filter column (index), so you can't pass lists in query_kwargs.

* Use 'earliest' & 'latest' time modifiers to customize the time range of your search. You can specify an exact time such as earliest="10/5/2019:20:00:00", or a relative time such as earliest=-1h or latest=@w6. To learn more about SPL TimeMoidifers see: https://docs.splunk.com/Documentation/Splunk/7.2.6/SearchReference/SearchTimeModifiers.

* It's recommended to declare selected columns in query_kwargs ('columns').

* You can limit the results number in query_kwargs ('max_columns').

* Note that no results might be a result of a syntax error in column name/filter value.   


## Step 6: Using loop for getting more the 50,000 results 

Splunk API limits the number of results per query to 50,000.

As for now, to overcome this limitation you can use the following 'for' loop.

**First, we will need to set a base query_kwargs.**

For this example, lets query SIRI for one high frequency bus route in Jerusaelm (Line Route 15 of Egged).

In [9]:
query_kwargs = {'index': 'siri',
                'agency_id': 3,
                'route_id': 12405,
               'columns': ['timestamp','agency_id','route_id','route_short_name','service_id','planned_start_time',
                        'bus_id','predicted_end_time','time_recorded','lat','lon']
                }

**Second, define time gaps (e.g days, hours, minutes) for the loop. The time gap need to be small enough for collecting no more then 50,000 results per run, and big enough for better performance.**

See example of quering for last 30 days data using time gaps of 6 days.

In [10]:
time_unit = 'd'
earliest_time_units = 30
latest_time_units = 0
time_units_gap = 6

earliest_latest =\
    [("-{}{}".format(u+time_units_gap, time_unit), "-{}{}".format(u, time_unit)) for u in range(int(round(latest_time_units,0)),
                                                                                               int(round(earliest_time_units,0)),
                                                                                                int(round(time_units_gap,0)))] 

print ("search from {} to {}".format(earliest_latest[0], earliest_latest[-1]))

search from ('-6d', '-0d') to ('-30d', '-24d')


In [11]:
%%time

import pandas as pd

SIRI_data = pd.DataFrame()

for earliest, latest in earliest_latest:
    
    query_kwargs['earliest'] = earliest
    query_kwargs['latest'] = latest
    
    data = read_splunk(splunk_query_builder(**query_kwargs),
                host =  HOST, port = PORT, username = USERNAME, password = PASSWORD)
        
    if isinstance(data, pd.DataFrame) and len(data)==50000:
        print ('\n\nCollecting results stopped!\n\n')
        break

    else:
        SIRI_data = SIRI_data.append(data)
            
SIRI_data.drop_duplicates(inplace=True)

start..

your query:
 search index=siri earliest=-6d latest=-0d agency_id=3 route_id=12405 |
 fields timestamp, agency_id, route_id, route_short_name, service_id, planned_start_time, bus_id, predicted_end_time, time_recorded, lat, lon |


connection succeed

query status: 100.0%   30906 scanned   30906 matched   30906 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 30906

start..

your query:
 search index=siri earliest=-12d latest=-6d agency_id=3 route_id=12405 |
 fields timestamp, agency_id, route_id, route_short_name, service_id, planned_start_time, bus_id, predicted_end_time, time_recorded, lat, lon |


connection succeed

query status: 100.0%   41201 scanned   41201 matched   41201 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 41201

start..

your query:
 search index=siri earliest=-18d latest=-12d agency_id=3 route_id=12405 |
 fields timestamp, agency_id, route_id

In [12]:
SIRI_data.head()

,timestamp,agency_id,route_id,route_short_name,service_id,planned_start_time,bus_id,predicted_end_time,time_recorded,lat,lon
0,2019-06-02T23:59:24.776,3,12405,15,39071029,23:58:00,7721969,00:19:00,23:56:45,31.784912,35.200710
1,2019-06-02T23:59:24.776,3,12405,15,39071028,23:45:00,7794669,00:19:00,23:58:57,31.786530,35.198864
2,2019-06-02T23:58:19.452,3,12405,15,39071029,23:58:00,7721969,00:19:00,23:56:45,31.784912,35.200710
3,2019-06-02T23:58:19.452,3,12405,15,39071028,23:45:00,7794669,00:19:00,23:58:36,31.786503,35.198841
4,2019-06-02T23:57:16.845,3,12405,15,39071029,23:58:00,7721969,00:19:00,23:56:45,31.784912,35.200710


In [13]:
len(SIRI_data)

204579